# Create the grid for Richards 1D

This code allows to create a mesh for 1D PDE problem:
    - domain discretization
    - parameters setting
All output data are stored in a NetCDF file

In [1]:
# -*- coding: utf-8 -*-
"""
  GNU GPL v3 License
 
  Copyright 2018 Niccolo` Tubini
 
  This program is free software: you can redistribute it and/or modify
  it under the terms of the GNU General Public License as published by
  the Free Software Foundation, either version 3 of the License, or
  (at your option) any later version.
 
  This program is distributed in the hope that it will be useful,
  but WITHOUT ANY WARRANTY; without even the implied warranty of
  MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
  GNU General Public License for more details.
 
  You should have received a copy of the GNU General Public License
  along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""


import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
os.chdir("C:\\Users\\Niccolo\\OMS\\OMS_Project_Richards1D\\data\\RichardMeshGen_input")
os.listdir()

['Casulli2010_test2.csv',
 'Casulli2010_test2VG.csv',
 'ClaySand_noPonding.csv',
 'ClaySand_noPonding.PNG',
 'Clay_noPonding.csv',
 'Clay_noPonding.PNG',
 'Clay_noPondingBC.csv',
 'Clay_noPonding_Dry.csv',
 'SandClay_01Ponding_Measurement.csv',
 'SandClay_01Ponding_Measurement.PNG',
 'SandClay_noPonding.csv',
 'Sand_01Ponding.csv',
 'Sand_01Ponding.PNG']

### Read the input file in format .csv

In [3]:

data = pd.read_csv("Clay_noPonding_Dry.csv")
#print(data.head())
#print(data.index)
#print(data.columns)

### This section computes geometrical quantities of the discretized domain
- eta is the vertical coordinate of the control volumes centroids. eta is positive upward and its origin is set at the soil surface. This coordinate will be used to compute intial condition and plot outputs such as water head and water content
- etaDual is the vertical coordiante of control volumes interfaces. etaDual is positive upward and its origin is set at the soil surface. This coordinate will be used to plot output velocities.
- z is the vertical coordinate of the control volumes centroids. z is positive upward and its origin is set at the bottom of the soil column.
- zDual is the vertical coordiante of control volumes interfaces. zDual is positive upward and its origin is set at the bottom of the soil column.

In [4]:

## list containing centroids coordinates
eta = [] 
## list containing control volumes interface coordinates
etaDual = []

    
for i in range(np.size(data.index)-1,0,-1):
    if data['Type'][i]=='L' and data['Type'][i-1]=='L':
        #print("Layer layer")
        deta = ( data['eta'][i]-data['eta'][i-1])/data['N'][i-1]
        eta=np.append(eta, np.linspace(data['eta'][i]-deta/2,data['eta'][i-1]+deta/2,num=data['N'][i-1],endpoint=True) )
        etaDual=np.append(etaDual, np.linspace(data['eta'][i],data['eta'][i-1],num=data['N'][i-1]+1,endpoint=True) )
    elif data['Type'][i]=='L' and data['Type'][i-1]=='M':
        #print("Layer Meas")
        deta = ( data['eta'][i]-data['eta'][i-1])/data['N'][i-1]
        eta=np.append(eta, np.linspace(data['eta'][i]-deta/2,data['eta'][i-1],num=data['N'][i-1],endpoint=True) )
        etaDual=np.append(etaDual, np.linspace(data['eta'][i],data['eta'][i-1]+deta/2,num=data['N'][i-1],endpoint=True) )
    elif data['Type'][i]=='M' and data['Type'][i-1]=='L':
        #print("Meas layer")
        deta = ( data['eta'][i]-data['eta'][i-1])/data['N'][i-1]
        eta=np.append(eta, np.linspace(data['eta'][i],data['eta'][i-1]+deta/2,num=data['N'][i-1],endpoint=True) )
        etaDual=np.append(etaDual, np.linspace(data['eta'][i]-deta/2,data['eta'][i-1],num=data['N'][i-1],endpoint=True) )
    else:
        print("ERROR!!")  
        
## to eliminate doubles
eta=[ii for n,ii in enumerate(eta) if ii not in eta[:n]]
etaDual=[ii for n,ii in enumerate(etaDual) if ii not in etaDual[:n]]


## control volume length
length = []

for i in range(0,np.size(etaDual)-1):
    length = np.append(length, np.abs(etaDual[i]-etaDual[i+1]) )

    
## space length: is used to cumpute gradients
spaceDelta = []

for i in range(0,np.size(etaDual)):
    if i==0:
        spaceDelta = np.append(spaceDelta, np.abs(etaDual[i]-eta[i]) )
    elif i==np.size(etaDual)-1:
        spaceDelta = np.append(spaceDelta, np.abs(etaDual[i]-eta[i-1]) )
    else:
         spaceDelta = np.append(spaceDelta, np.abs(eta[i-1]-eta[i]) )
   

eta= np.append(eta,data['eta'][0])


In [5]:
## DA AGGIUNGERE ALLA FINE PERCHE SONO QUELLE CHE SERVONO IN ECLIPSE
z = []
zDual = []
for i in range(0, np.size(eta)):
    z = np.append(z,eta[i]-data['eta'][np.size(data['eta'])-1])
    
for i in range(0, np.size(etaDual)):
    zDual = np.append(zDual,etaDual[i]-data['eta'][np.size(data['eta'])-1])

## Grid plot
The sections below allows to plot the discretized domain. In the first plot control volumes centroids are shown, whilst in the second one their interfaces. Moreover, in both plots soil layer and measurament points are drawn.

These plots are drawn with bokeh library https://bokeh.pydata.org/en/latest/, and they are interactive.

In [6]:

# Standard imports 
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import BoxSelectTool
from bokeh.models import HoverTool

output_notebook()


labelSize = 18
titleSize = 22
legendSize = 14
axisTicksSize = 14

lineWidth =1.5

Loading BokehJS ...

In [7]:
x=np.zeros(np.size(z))

hover = HoverTool(tooltips=[
    ("(x,y)", "($x, $y)"),
])

p1 = figure(plot_width=600, plot_height=600,tools=['pan,wheel_zoom,box_zoom,reset',hover],
           title="Mouse over the dots",x_range=(-0.7, 0.7))
p1.scatter(x,eta, color="blue", marker='circle',legend='Centroids')

for i in range(0,np.size(data.index)):
    if data['Type'][i] == 'L':
        c = 'red'
        l = 'layer'
    elif data['Type'][i] == 'M':
        c = 'green'
        l = 'meas. point'
        
    p1.line([-0.2,0.2], [data['eta'][i],data['eta'][i]], color=c,line_width=lineWidth,legend=l)
p1.line([-0.2,0.2], [data['eta'][data['eta'].size-1],data['eta'][data['eta'].size-1]], color="red",line_width=lineWidth,legend='layer')

p1.yaxis.axis_label = '\u03b7 [m]'
p1.yaxis.axis_label_text_font_size = str(labelSize) + "px"
p1.title.text = 'Control volume centroids'
p1.title.align = "center"
p1.title.text_font_size = str(titleSize) + "px"

p1.legend.location = "top_left"
p1.legend.label_text_font_size = str(legendSize) + "px"
p1.legend.click_policy="hide"

show(p1)

In [8]:
## graphical representation of your grid

x=np.zeros(np.size(zDual))

hover = HoverTool(tooltips=[
    ("(x,y)", "($x, $y)"),
])

p1 = figure(plot_width=600, plot_height=600,tools=['pan,wheel_zoom,box_zoom,reset',hover],
           title="Mouse over the dots",x_range=(-0.7, 0.7))
p1.scatter(x,etaDual, color="blue", marker='cross',legend='CV interface')
for i in data.index:
    if data['Type'][i] == 'L':
        c = 'red'
        l = 'layer'
    elif data['Type'][i] == 'M':
        c = 'green'
        l = 'meas. point'
        
    p1.line([-0.2,0.2], [data['eta'][i],data['eta'][i]], color=c,line_width=lineWidth,legend=l)
p1.line([-0.2,0.2], [data['eta'][data['eta'].size-1],data['eta'][data['eta'].size-1]], color="red",line_width=lineWidth,legend='layer')

p1.yaxis.axis_label = '\u03b7 [m]'
p1.yaxis.axis_label_text_font_size = str(labelSize) + "px"
p1.title.text = 'Control volume intefaces'
p1.title.align = "center"
p1.title.text_font_size = str(titleSize) + "px"

p1.legend.location = "top_left"
p1.legend.label_text_font_size = str(legendSize) + "px"
p1.legend.click_policy="hide"

show(p1)

## Compute initial condition
This section allows to compute the initial condition for the water suction. It is possible to define the initial condition in three ways:
- hydrostatic distribution for a prescribed $\psi$ @bottom;
- linear interpolation of measuraments points and $\psi$ @bottm and @surface;
- constant hydraulic head for a prescribed $\psi$ @bottom

It is important to note that at the surface the hydraulic head is prescribed in the inputFile.csv. If it is positive means there is ponding water otherwise the matric suction of the uppermost layer of the soil.


In [9]:
psiIC = []
coordMeasPoint = []
psiMeas = []

for i in data.index:
    if data['Type'][i] == 'M':
        coordMeasPoint.append(data['eta'][i])
        psiMeas.append(data['psi'][i])
        
coordMeasPoint = np.append(coordMeasPoint, data['eta'][np.size(data['eta'])-1])
psiMeas = np.append(psiMeas,data['psi'][np.size(data['psi'])-1])

In [10]:
coordMeasPoint

array([-2.])

In [11]:
psiMeas

array([-50.])

#### Hydrostatic distribution for a given psi at the bottom

In [27]:

for i in range(0,np.size(eta)-1):
    psiIC = np.append(psiIC,data['psi'][np.size(data['psi'])-1]+(data['eta'][np.size(data['eta'])-1]-eta[i]))

psiIC = np.append(psiIC,data['psi'][0] )

In [28]:


p1 = figure(plot_width=600, plot_height=600,tools=['pan,wheel_zoom,box_zoom,reset',hover],
           title="Mouse over the dots")
p1.scatter(psiIC,eta, color="blue")

p1.xaxis.axis_label = '\u03C8 [m]'
p1.xaxis.axis_label_text_font_size = str(labelSize) + "px"

p1.yaxis.axis_label = '\u03b7 [m]'
p1.yaxis.axis_label_text_font_size = str(labelSize) + "px"
p1.title.text = 'Initial condition for \u03C8: hydrostatic'
p1.title.align = "center"
p1.title.text_font_size = str(titleSize) + "px"


show(p1)

The _full_ hydrostatic distribution is the state of equilibrium: the water flux is due to the spatial gradient of $\psi+z$. Let us plot the total hydraulic head for the hydrostatic distribution and it is possible to observe that the spatial gradient is null every where:

In [45]:


p1 = figure(plot_width=600, plot_height=600,tools=['pan,wheel_zoom,box_zoom,reset',hover],
           title="Mouse over the dots")
p1.scatter(psiIC+z,eta, color="blue")

p1.xaxis.axis_label = '\u03C8 [m]'
p1.xaxis.axis_label_text_font_size = str(labelSize) + "px"

p1.yaxis.axis_label = '\u03b7 [m]'
p1.yaxis.axis_label_text_font_size = str(labelSize) + "px"
p1.title.text = 'Total hydraulic head'
p1.title.align = "center"
p1.title.text_font_size = str(titleSize) + "px"


show(p1)

#### Constant pressure head distribution for a given psi at the bottom


In [12]:
for i in range(0,np.size(eta)-1):
    psiIC = np.append(psiIC,data['psi'][np.size(data['psi'])-1])

psiIC = np.append(psiIC,data['psi'][0] )

In [13]:

p1 = figure(plot_width=600, plot_height=600,tools=['pan,wheel_zoom,box_zoom,reset',hover],
           title="Mouse over the dots")
p1.scatter(psiIC,eta, color="blue")

p1.xaxis.axis_label = '\u03C8 [m]'
p1.xaxis.axis_label_text_font_size = str(labelSize) + "px"

p1.yaxis.axis_label = '\u03b7 [m]'
p1.yaxis.axis_label_text_font_size = str(labelSize) + "px"
p1.title.text = 'Initial condition for \u03C8: constant'
p1.title.align = "center"
p1.title.text_font_size = str(titleSize) + "px"


show(p1)

#### Linear interpolation of measuraments points

In [49]:

for i in range(np.size(coordMeasPoint)-1,-1,-1):
    etaInterp = []
    if i==np.size(coordMeasPoint)-1:
        etaInterp = eta[0:eta.tolist().index(coordMeasPoint[i-1])]
        etap = [coordMeasPoint[i],coordMeasPoint[i-1]]
        fp = [psiMeas[i],psiMeas[i-1]]
        psiIC = np.append(psiIC, np.interp(etaInterp,etap,fp) )
        #for j in range(0,np.size(etaInterp)):
        #    psiIC = np.append(psiIC,psiMeas[i]+(coordMeasPoint[i]-etaInterp[j]))
    elif i== 0:
        etaInterp = eta[eta.tolist().index(coordMeasPoint[i]):np.size(eta)-1]
        etap = [coordMeasPoint[i],0]
        #fp = [psiMeas[i],data['psi'][0]]
        fp = [psiMeas[i],-z[np.size(z)-1]]
        psiIC = np.append(psiIC, np.interp(etaInterp,etap,fp) )
    else:
        etaInterp = eta[eta.tolist().index(coordMeasPoint[i-1]):eta.index(coordMeasPoint[i])]
        etap = [coordMeasPoint[i],coordMeasPoint[i-1]]
        fp = [psiMeas[i],psiMeas[i-1]]
        psiIC = np.append(psiIC, np.interp(etaInterp,etap,fp) )

psiIC = np.append(psiIC,data['psi'][0] )

 

In [50]:
p1 = figure(plot_width=600, plot_height=600,tools=['pan,wheel_zoom,box_zoom,reset',hover],
           title="Mouse over the dots")
p1.scatter(psiIC,eta, color="blue")

p1.xaxis.axis_label = '\u03C8 [m]'
p1.xaxis.axis_label_text_font_size = str(labelSize) + "px"

p1.yaxis.axis_label = '\u03b7 [m]'
p1.yaxis.axis_label_text_font_size = str(labelSize) + "px"
p1.title.text = 'Initial condition for \u03C8:\n interpolated from measuraments'
p1.title.align = "center"
p1.title.text_font_size = str(titleSize) + "px"


show(p1)  

## Set parameters
This section assing to each control volume of the domain its SWRC parameters accordingly with its depth

In [14]:
thetaS = []
thetaR = []
Ks = []
par1SWRC = []
par2SWRC = []
par3SWRC = []
par4SWRC = []
et = []


coordLayer = []
tempThetaS = []
tempThetaR = []
tempKs = []
tempPar1 = []
tempPar2 = []
tempPar3 = []
tempPar4 = []
tempEt = []

for i in data.index:
    if data['Type'][i] == 'L':
        coordLayer.append(data['eta'][i])
        tempThetaS.append(data['thetaS'][i])
        tempThetaR.append(data['thetaR'][i])
        tempKs.append(data['Ks'][i])
        tempPar1.append(data['par1SWRC'][i])
        tempPar2.append(data['par2SWRC'][i])
        tempPar3.append(data['alphaSpecificStorage'][i])
        tempPar4.append(data['betaSpecificStorage'][i])
        tempEt.append(data['et'][i])

In [15]:

for i in range(np.size(coordLayer)-1,0,-1):
    for j in range(0,np.size(eta)):
        if(eta[j]>coordLayer[i] and eta[j]<coordLayer[i-1] ):
            thetaS.append(tempThetaS[i-1])
            thetaR.append(tempThetaR[i-1])
            Ks.append(tempKs[i-1])
            par1SWRC.append(tempPar1[i-1])
            par2SWRC.append(tempPar2[i-1])
            par3SWRC.append(tempPar3[i-1])
            par4SWRC.append(tempPar4[i-1])
            et.append(tempEt[i-1])
    
## add et coeff for free water
et = np.append(et,0)

## Grid plot
The section below allows to plot the distribution of SWRC parameters in the soil column. In the first plot control volumes centroids are shown, whilst in the second one their interfaces. Moreover, in both plots soil layer and measurament points are drawn.

These plots are drawn with bokeh library https://bokeh.pydata.org/en/latest/, and they are interactive.

In [16]:

hover = HoverTool(tooltips=[
    ("(x,y)", "($x, $y)"),
])

p1 = figure(plot_width=600, plot_height=600,tools=['pan,wheel_zoom,box_zoom,reset',hover],
           title="Mouse over the dots")

p1.scatter(thetaS, eta[0:np.size(eta)-1], color="blue", line_width=lineWidth)
p1.xaxis.axis_label = ' water content at saturation [-]'
p1.xaxis.axis_label_text_font_size = str(labelSize) + "px"
p1.yaxis.axis_label = '\u03b7 [m]'
p1.yaxis.axis_label_text_font_size = str(labelSize) + "px"
p1.title.text = 'Water content at saturation'
p1.title.align = "center"
p1.title.text_font_size = str(titleSize) + "px"
tab1 = Panel(child=p1, title="Theta_s")

p2 = figure(plot_width=600, plot_height=600,tools=['pan,wheel_zoom,box_zoom,reset',hover],
           title="Mouse over the dots")
p2.scatter(thetaR, eta[0:np.size(eta)-1],line_width=lineWidth, color="red")
p2.xaxis.axis_label = 'residual water content [-]'
p2.xaxis.axis_label_text_font_size = str(labelSize) + "px"
p2.yaxis.axis_label = '\u03b7 [m]'
p2.yaxis.axis_label_text_font_size = str(labelSize) + "px"
p2.title.text = 'Residual water content'
p2.title.align = "center"
p2.title.text_font_size = str(titleSize) + "px"
tab2 = Panel(child=p2, title="Theta_r")

p3 = figure(plot_width=600, plot_height=600,tools=['pan,wheel_zoom,box_zoom,reset',hover],
           title="Mouse over the dots" )
p3.scatter(Ks, eta[0:np.size(eta)-1],line_width=lineWidth, color="red")
p3.xaxis.axis_label = 'Ks [m/s]'
p3.xaxis.axis_label_text_font_size = str(labelSize) + "px"
p3.yaxis.axis_label = '\u03b7 [m]'
p3.yaxis.axis_label_text_font_size = str(labelSize) + "px"
p3.title.text = 'Saturated hydraulic conductivity'
p3.title.align = "center"
p3.title.text_font_size = str(titleSize) + "px"
tab3= Panel(child=p3, title="Ks")

p4 = figure(plot_width=600, plot_height=600,tools=['pan,wheel_zoom,box_zoom,reset',hover],
           title="Mouse over the dots" )
p4.scatter(par1SWRC, eta[0:np.size(eta)-1],line_width=lineWidth, color="red")
p4.xaxis.axis_label = ' '
p4.xaxis.axis_label_text_font_size = str(labelSize) + "px"
p4.yaxis.axis_label = '\u03b7 [m]'
p4.yaxis.axis_label_text_font_size = str(labelSize) + "px"
p4.title.text = 'SWRC parameter 2'
p4.title.align = "center"
p4.title.text_font_size = str(titleSize) + "px"
tab4= Panel(child=p4, title="SWRC par1")

p5 = figure(plot_width=600, plot_height=600,tools=['pan,wheel_zoom,box_zoom,reset',hover],
           title="Mouse over the dots" )
p5.scatter(par2SWRC, eta[0:np.size(eta)-1],line_width=lineWidth, color="red")
p5.xaxis.axis_label = ' '
p5.xaxis.axis_label_text_font_size = str(labelSize) + "px"
p5.yaxis.axis_label = '\u03b7 [m]'
p5.yaxis.axis_label_text_font_size = str(labelSize) + "px"
p5.title.text = 'SWRC parameter 2'
p5.title.align = "center"
p5.title.text_font_size = str(titleSize) + "px"
tab5= Panel(child=p5, title="SWRC par2")

p6 = figure(plot_width=600, plot_height=600,tools=['pan,wheel_zoom,box_zoom,reset',hover],
           title="Mouse over the dots" )
p6.scatter(par3SWRC, eta[0:np.size(eta)-1],line_width=lineWidth, color="red")
p6.xaxis.axis_label = '\u03b1 [1/Pa]'
p6.xaxis.axis_label_text_font_size = str(labelSize) + "px"
p6.yaxis.axis_label = '\u03b7 [m]'
p6.yaxis.axis_label_text_font_size = str(labelSize) + "px"
p6.title.text = 'Compressibility of soil'
p6.title.align = "center"
p6.title.text_font_size = str(titleSize) + "px"
tab6= Panel(child=p6, title="\u03b1")

p7 = figure(plot_width=600, plot_height=600,tools=['pan,wheel_zoom,box_zoom,reset',hover],
           title="Mouse over the dots" )
p7.scatter(par4SWRC, eta[0:np.size(eta)-1],line_width=lineWidth, color="red")
p7.xaxis.axis_label = '\u03b2 [1/Pa]'
p7.xaxis.axis_label_text_font_size = str(labelSize) + "px"
p7.yaxis.axis_label = '\u03b7 [m]'
p7.yaxis.axis_label_text_font_size = str(labelSize) + "px"
p7.title.text = 'Compressibility of water'
p7.title.align = "center"
p7.title.text_font_size = str(titleSize) + "px"
tab7= Panel(child=p7, title="\u03b2")

p8 = figure(plot_width=600, plot_height=600,tools=['pan,wheel_zoom,box_zoom,reset',hover],
           title="Mouse over the dots" )
p8.scatter(et, eta,line_width=lineWidth, color="red")
p8.xaxis.axis_label = 'et coeff. [1/s]'
p8.xaxis.axis_label_text_font_size = str(labelSize) + "px"
p8.yaxis.axis_label = '\u03b7 [m]'
p8.yaxis.axis_label_text_font_size = str(labelSize) + "px"
p8.title.text = 'Source sink term'
p8.title.align = "center"
p8.title.text_font_size = str(titleSize) + "px"
tab8= Panel(child=p8, title="et")

tabs = Tabs(tabs=[ tab1, tab2, tab3, tab4, tab5, tab6, tab7, tab8 ])
show(tabs)

## Save data in a NetCDF

All data computed above are saved in NetCDF file

In [17]:
## enter the following string to write your NetCDF

fileName = "C:\\Users\\Niccolo\\OMS\\OMS_Project_Richards1D\\data\\Grid_NetCDF\\Clay_noPonding_Dry.nc"

## file attributes

title = '''Grid 1 layer clay no ponding water, constant psi with -50m at bottom, 
            clay thetaS =  0.5, thetaR = 0.07, Ks = 0.00000023m/s, alphaSpecificStorage = 0, betaSpecificStorage = 0
                  VG model: n = 1.16, alpha = 5.58m 
            inputfile: data\\RichardMeshGen_input\\Clay_noPonding_Dry.csv'''
institution = 'Geoframe'
summary = 'This file stores all grid information (geometry, soiAll data computed above are saved in NetCDF file'
date = '5/23/2018' 


In [18]:

from netCDF4 import Dataset



# the output array to write will be nx x ny
dim = np.size(eta);
dim1 = np.size(thetaS)
# open a new netCDF file for writing.
ncfile = Dataset(fileName,'w') 

# Create global attributes
ncfile.title = title
ncfile.institution =  institution
ncfile.summary = summary
#ncfile.acknowledgment = ""
ncfile.date_created = date


# create the z dimensions.
ncfile.createDimension('z',dim)
ncfile.createDimension('zz',dim1)

# create the variable
# first argument is name of variable, second is datatype, third is
# a tuple with the names of dimensions.
dataEta = ncfile.createVariable('eta','f8',('z'))
dataEta.unit = 'm'
dataEta.long_name = '\u03b7 coordinate of volume centroids: zero is at soil surface and and positive upward'

dataEtaDual = ncfile.createVariable('etaDual','f8',('z'))
dataEtaDual.unit = 'm'
dataEtaDual.long_name = '\u03b7 coordinate of volume interfaces: zero is at soil surface and and positive upward. '

dataZ = ncfile.createVariable('z','f8',('z'))
dataZ.unit = 'm'
dataZ.long_name = 'z coordinate  of volume centroids: zero is at the bottom of the column and and positive upward'

dataZDual = ncfile.createVariable('zDual','f8',('z'))
dataZDual.unit = 'm'
dataZDual.long_name = 'z coordinate of volume interfaces: zero is at soil surface and and positive upward.'


dataPsiIC = ncfile.createVariable('psiIC','f8',('z'))
dataPsiIC.units = 'm'
dataPsiIC.long_name = 'initial condition for water suction'

dataSpaceDelta = ncfile.createVariable('spaceDelta','f8',('z'))
dataSpaceDelta.unit = 'm'
dataSpaceDelta.long_name = 'Distance between consecutive controids, is used to compute gradients'

dataEt = ncfile.createVariable('et','f8',('z'))
dataEt.units = '1/s'
dataEt.long_name = 'Coefficient to simulate ET'




dataDeltaZ = ncfile.createVariable('deltaZ','f8',('zz'))
dataDeltaZ.unit = 'm'
dataDeltaZ.long_name = 'Length of each control volume'

dataThetaS = ncfile.createVariable('thetaS','f8',('zz'))
dataThetaS.units = '-'
dataThetaS.long_name = 'adimensional water content at saturation'

dataThetaR = ncfile.createVariable('thetaR','f8',('zz'))
dataThetaR.units = '-'
dataThetaR.long_name = 'adimensional residual water content'

dataKs = ncfile.createVariable('Ks','f8',('zz'))
dataKs.units = 'm/s'
dataKs.long_name = 'hydraulic conductivity at saturation'

## enter the long_name of par1SWRC, as an example  Parameter n of Van Genuchten model
dataPar1SWRC = ncfile.createVariable('par1SWRC','f8',('zz'))
dataPar1SWRC.units = '-'
dataPar1SWRC.long_name = 'Parameter n of Van Genuchten model'#'Parameter n of Brooks and Corey model'#

## enter the long_name of par2SWRC, as an example  Parameter alpha of Van Genuchten model
dataPar2SWRC = ncfile.createVariable('par2SWRC','f8',('zz'))
dataPar2SWRC.units = 'm'
dataPar2SWRC.long_name = 'Parameter alpha of Van Genuchten model'#'Parameter psiD of Brooks and Corey model'#

dataPar3SWRC = ncfile.createVariable('par3SWRC','f8',('zz'))
dataPar3SWRC.units = '1/Pa'
dataPar3SWRC.long_name = 'Aquitard compressibility'

dataPar4SWRC = ncfile.createVariable('par4SWRC','f8',('zz'))
dataPar4SWRC.units = '1/Pa'
dataPar4SWRC.long_name = 'Water compressibility'



# write data to variable.
for i in range(0,dim):
    dataEta[i] = eta[i]
    dataEtaDual[i] = etaDual[i]
    dataZ[i] = z[i]
    dataZDual[i] = zDual[i]
    dataPsiIC[i] = psiIC[i]
    dataSpaceDelta[i] = spaceDelta[i]
    dataEt[i] = et[i]
   

for i in range(0,dim1):
    dataDeltaZ[i] = length[i]
    dataThetaS[i] = thetaS[i]
    dataThetaR[i] = thetaR[i]
    dataKs[i] = Ks[i]
    dataPar1SWRC[i] = par1SWRC[i]
    dataPar2SWRC[i] = par2SWRC[i]
    dataPar3SWRC[i] = par3SWRC[i]
    dataPar4SWRC[i] = par4SWRC[i]
   
    
## close the file.
ncfile.close()
print ('*** SUCCESS writing!  '+fileName)


*** SUCCESS writing!  C:\Users\Niccolo\OMS\OMS_Project_Richards1D\data\Grid_NetCDF\Clay_noPonding_Dry.nc
